## Read image data

In [ ]:
import os
import pprint
import json
from typing import Any, Dict, Union
from copy import deepcopy
import sqlite3


from lib.comfy_analysis import ComfyImage, fileToComfyImage, SchemaNode, Schema3, extract_positive_prompt
# from lib.comfy_analysis import Schema2, get_parent, prompt_schedule_gen05, prompt_schedule_gen04

pp = pprint.PrettyPrinter()

# Test single image

In [ ]:

# TODO: Use sqlite dump
data_path = 'data/'

images = [img for img in os.listdir(data_path) if img[-4:] == '.png']

print(f"Processing {len(images)} images")
ImageObjects = [fileToComfyImage(data_path + img) for img in images]


In [ ]:
img = [img for img in ImageObjects if 'Dragon' in img.filename][0]

# img.workflow.keys()
# Workflow keys 'last_node_id', 'last_link_id', 'nodes', 'links', 'groups', 'config', 'extra', 'version', 'widget_idx_map', 'seed_widgets'


# Workflow contains all nodes with ui props like color, size, position, as well as node properties.
# 85 nodes in this thing.
# len(img.workflow['nodes'])
img.workflow['nodes']


# 48 nodes in prompt. Stripped down, map from ID to a struct of values. IDs are identical.
# Contains title in metadata '_meta': {'title': 'Negative'}},
# 'class_type' gives type
img.prompt

# Run through sqlite db

In [ ]:
from typing import List

sqlite_db = '/home/abrahams/training_toolbox/golang/internal/testdata/prompts.sqlite'

conn = sqlite3.connect(sqlite_db)
# Iterate over prompt strings in db

BadImages: List[ComfyImage] = []
positives: List[str] = []

for row in conn.execute('SELECT file_path, prompt FROM prompts'):
    try:
        prompt = json.loads(row[1])
        filename = row[0]
        img = ComfyImage(filename, prompt, {})
        positive = extract_positive_prompt(prompt)
        positives.append(positive)
    except Exception:
        BadImages.append(img)
        pass

print(f"{len(positives)} / {len(positives)+len(BadImages)}")


In [ ]:
print(BadImages[0].filename)
# print(json.dumps(BadImages[0].prompt, indent=2))

# extract_positive_prompt(BadImages[0].prompt)


# Play with prompts

In [ ]:
from collections import Counter
import random

special_tags = ['source_furry', 'score_9', 'score_8_up', 'score_7_up', 'score_6_up', 'score_5_up', 'score_4_up', 'rating_explicit', 'rating_questionable', 'rating_safe',
               'illustration', 'digital illustration art', 'official art', 'edit',
                'masterpiece', 'best quality', 'absurdres', 'hires', 'hi res', 'very awa', 'very aesthetic', 'newest', '()']
def clean_prompt(prompt: str) -> str:
    prompt = prompt.lower()
    for tag in special_tags:
        prompt = prompt.replace(tag + ', ', '').replace(tag, '')

    prompt = prompt.strip().replace('\n', ' ').replace('\r', ' ')
    return prompt

prompts = Counter([clean_prompt(p) for p in positives])



print(len(prompts))

In [ ]:
random.choices(list(prompts), k=100)

## Schemas


# Analyze Loras

In [67]:
# LoRA Regex, matches strings as follows
# <lora:TeradaOchiko.safetensors:1.0>
# <lora:Personal/Hex/HexXL_v5_FewerYellow.safetensors:0.5>
import re
from collections import Counter
loraMatch = re.compile(r'<lora:(.*?):([0-9\.]+)>')


loraCounter = Counter()

for img in ImageObjects:
    item340 = img.prompt.get('340', None)
    if item340 is None:
        print("Unable to find node 340 in " + img.filename)
        continue
    if item340['class_type'] != 'Power Prompt (rgthree)':
        print("Unexpected node type: ")
        print(img.filename)
    pprompt = item340['inputs']['prompt']
    # Match Loras
    loras = loraMatch.findall(pprompt)
    for l in loras:
        lora_name = l[0].replace("\\", "/").split('/')[-1].replace('.safetensors', '')
        loraCounter[lora_name] += 1


In [70]:

# Build orderedDictionary from loraCounter Counter object()
loraCounterOut = dict(loraCounter.most_common())

with open('data/loraCounter_damn.json', 'w') as f:
    # Output json dictionary by top counts
    json.dump(loraCounterOut, f, indent=4)

In [4]:

import os
import pprint
pp = pprint.PrettyPrinter()
import json
from collections import Counter

loraCounter = Counter()

for path in os.listdir('data'):
    if path[-5:] == '.json':
        with open('data/' + path) as f:
            data = json.load(f)
            for lora, value in data.items():
                loraCounter[lora] += value


In [ ]:
loraCounter

In [12]:

targets = {
    'Sirius': [['Sirius', 'bl4ckdr4g'], {}],
    'Bats': [['Bats', 'Aerys', 'TanBat'], {}],
    'Hex': [['Hex'], {}],
    'PixelSketcher': [['PixelSketcher'], {}],
    'Valziel': [['Valziel'], {}],
    'Zaush': [['Zaush'], {}],
    'Tricksta': [['Tricksta'], {}],
}



for name, matchData in targets.items():
    for loraName, count in loraCounter.items():
        if any([loraName.find(match) > -1 for match in matchData[0]]):
            matchData[1][loraName] = count
    matchData[1] = dict(sorted(matchData[1].items(), key = lambda item: item[1], reverse=True))






In [ ]:
targets